In [76]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter
import statsmodels.formula.api as smf

In [77]:
city = 'CMX'
    
dias = 0
#year-month-day
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
#non_buyers_date = '2022-08-26' #1 months
customer_date_desde_query = (datetime.datetime.today() - datetime.timedelta(days=30)).strftime('%Y-%m-%d') #1 months

customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=200))#.strftime('%Y-%m-%d') #1 months

In [78]:
query = """
-- 93058
-- Se toma como base el query 2718
WITH 
ful AS (
SELECT DISTINCT
    bo.order_id,
    DATE(bo.submit_date) as submit_date,
    ffg.close_date
    
FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group" bfg        ON bfg.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order" bfo        ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group" ffg         ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order" fo                      ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment" bop            ON bop.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" s                       ON s.site_id = bo.site_disc
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type" fot                ON fot.fb_order_type_id=fo.fb_order_type_id

WHERE fo.fb_order_status_id IN (1,6,7,8)
    AND bo.submit_date >= DATE_TRUNC('day', date(getdate()) - interval '1 day')
    AND bo.submit_date < DATE_TRUNC('day', date(getdate()) - interval '0 day')
    AND ffg.close_date >= DATE_TRUNC('day', date(getdate()) + interval '1 day')
    AND bo.order_status = 'SUBMITTED'
    AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
    AND bop.archived = 'N'
    AND (fot.name IS NULL OR fot.name <> 'REFUND')
    AND s.site_identifier_value = '{city}'
),

view_sku AS (
SELECT DISTINCT
  --ds.identifier_value,
  s.site_identifier_value,
  --ea.ctsessionid,
  ea.profile_identity,
  ea.frproductid,
  bs2.name,
  bcat2.name AS category,
  bcat.name AS subcat,
  --ea.frproductname,
  --ea.frproductinstock,
  DATE_TRUNC('day', ea.event_date) AS event_date
  --ea.eventname,
  --ea.frrecommendationtype,
  --ea.frfromproductid
  --ea.frscreen--,
  --ea.frsubmitresult--,--para el cart add
  --info_subs.num_substitutes
FROM dpr_clevertap.events_analytics                                     ea
INNER JOIN dpr_clevertap.user_profile                                   p       ON ea.profile_identity = p.profile_identity AND ea.profile_identity NOT ILIKE '%,%'
INNER JOIN dpr_shared.dim_site                                          ds      ON ds.identifier_value = p.registeredcity
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"          bp      ON bp.product_id::varchar = ea.frproductid::varchar
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"             s       ON s.site_id = bp.catalog_disc AND ds.identifier_value = s.site_identifier_value
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"              bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"         bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"    bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"         bcat2   ON bcx.category_id = bcat2.category_id

WHERE
  ea.eventname = 'FRF Product View'
  AND ea.event_date >= DATE_TRUNC('day', date(getdate()) - interval '1 day')
  AND ea.event_date < DATE_TRUNC('day', date(getdate()) - interval '0 day')
  AND ds.identifier_value = '{city}'
 ),
 
almost AS (

SELECT DISTINCT
    s.site_identifier_value AS city,
    bo.customer_id,
    ful.submit_date,
    ful.close_date,
    bs.addl_product_id as padre_sku_id
    
FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     s       ON s.site_id = bo.site_disc
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"               boi     ON boi.order_id=bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                foi     ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                     fo      ON fo.order_id = bo.order_id
INNER JOIN ful                                                                          ON ful.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"      bdoi    ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs      ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                  bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"            bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat2   ON bcx.category_id = bcat2.category_id

WHERE fb_order_status_id IN (1,6,7,8)
    AND bo.order_status = 'SUBMITTED'
   -- FIX SUPER DESCUENTOS
    AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
    AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
    AND bo.customer_id IN (SELECT DISTINCT profile_identity FROM view_sku)
    --AND bo.submit_date >= DATE_TRUNC('day', date(getdate()) - interval '1 day')
    --AND bo.submit_date < DATE_TRUNC('day', date(getdate()) - interval '0 day')
    
),

ordered AS (
SELECT DISTINCT
    a.city,
    a.customer_id,
    a.close_date
    
FROM almost a
)

SELECT DISTINCT
    vs.site_identifier_value AS city,
    vs.profile_identity AS customer_id,
    vs.frproductid AS card_id,
    vs.name AS product,
    vs.category AS cat,
    CASE
        WHEN vs.subcat ILIKE 'Pollo%' THEN 'Pollo'
        WHEN vs.subcat ILIKE 'Res%' THEN 'Res'
        WHEN vs.subcat ILIKE 'Cerdo%' THEN 'Cerdo'
        ELSE vs.subcat
    END AS subcat,
    vs.event_date,
    o.close_date
    
FROM view_sku vs
LEFT JOIN almost    a ON a.customer_id = vs.profile_identity AND a.city = vs.site_identifier_value AND a.padre_sku_id = vs.frproductid 
INNER JOIN ordered  o ON o.customer_id = vs.profile_identity AND o.city = vs.site_identifier_value

WHERE a.customer_id IS NULL
""".format(city=city)

df_views = read_connection_data_warehouse.runQuery(query)
df_views[["card_id"]] = df_views[["card_id"]].astype('int64')

In [79]:
frida_products = get_fresh_query_result("https://internal-redash.federate.frubana.com/",118629,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [80]:
df_margin = get_fresh_query_result("https://internal-redash.federate.frubana.com/",144521,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [81]:
df_margin = df_margin.loc[df_margin['day'] == df_margin['day'].max()].copy()
df_margin['margin'] = (df_margin['margin'].round(3)).astype(float)

In [82]:
query = """
-- 93058
-- Se toma como base el query 2718
WITH 
ful AS (

SELECT DISTINCT
    bo.order_id,
    bo.submit_date
    
FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group" bfg        ON bfg.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order" bfo        ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group" ffg         ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order" fo                      ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment" bop            ON bop.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" s                       ON s.site_id = bo.site_disc
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type" fot                ON fot.fb_order_type_id=fo.fb_order_type_id

WHERE fo.fb_order_status_id IN (1,6,7,8)
    AND ffg.close_date >= DATE_TRUNC('month', date(getdate()) - interval '1 month')
    AND bo.order_status = 'SUBMITTED'
    AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
    AND bop.archived = 'N'
    AND (fot.name IS NULL OR fot.name <> 'REFUND')
    AND s.site_identifier_value = '{city}'
)--,

--more AS (
SELECT DISTINCT
    s.site_identifier_value AS city,
    --bo.customer_id,
    --DATE(bo.submit_date) AS submit_date,
    bs.addl_product_id AS card_id,
    --bs2.name,
    --COUNT(DISTINCT bo.order_id) AS frequency,
    SUM(boipd.quantity * foi.step_unit * boi.sale_price)*1.0 AS gmv_usd--,
    --SUM(boipd.quantity*foi.step_unit) AS demand

FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     s       ON s.site_id = bo.site_disc
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"               boi     ON boi.order_id=bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                foi     ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                     fo      ON fo.order_id = bo.order_id
INNER JOIN ful                                                                          ON ful.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"      bdoi    ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs      ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                  bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"            bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat2   ON bcx.category_id = bcat2.category_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"      boipd   ON boipd.order_item_id=boi.order_item_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"        boida   ON boida.order_item_price_dtl_id=boipd.order_item_price_dtl_id

WHERE fb_order_status_id IN (1,6,7,8)
    AND bo.order_status = 'SUBMITTED'
   -- FIX SUPER DESCUENTOS
    AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
    AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
    AND bs.addl_product_id IN {cards} 
GROUP BY 1,2--,3,4
""".format(cards=tuple(set(df_views.card_id.unique())), city = city)

data_ventas_aux = read_connection_data_warehouse.runQuery(query)
data_ventas_aux[["gmv_usd"]] = data_ventas_aux[["gmv_usd"]].astype('float32')

In [83]:
df = pd.merge(df_views, data_ventas_aux, left_on=['city','card_id'], right_on=['city','card_id'], how='inner')
df = pd.merge(df, df_margin[['margin','id_tarjeta']], left_on='card_id', right_on='id_tarjeta', how='inner')

df['cash_margin'] = df.gmv_usd*df.margin

In [84]:
result_df = df.groupby('customer_id').agg(card_id=('card_id', list),
                                    product=('product', list),
                                    gmv_usd=('gmv_usd', list),
                                    cash_margin=('cash_margin', np.sum),
                                    gmv_usd_s=('gmv_usd', np.sum)
                                    ).reset_index()

In [97]:
result_df['margin'] = (result_df.cash_margin/result_df.gmv_usd_s)*100.00

result_df['discount'] =  np.where(result_df.margin > 10, 10,0.7*result_df.margin)
result_df['discount'] = result_df['discount'].apply(lambda x: round(x, 2))

result_df['offer_name'] = ["SH_AOV_VIEW_"+city+"_"+todays_date+"_"+str(customer) for customer in result_df.customer_id]

result_df = result_df.loc[result_df.discount > 1].reset_index(drop=True).copy()

In [86]:
def concatenate_skus(card_id_list):
    result = []
    for sku in card_id_list:
        try:
            sku_value = frida_products.loc[frida_products['id_tarjeta'] == sku, 'skus'].values[0]
            result.append(sku_value)
        except IndexError:
            result.append('')
    return '#'.join(result)

In [87]:
result_df.head(2)

,customer_id,card_id,product,gmv_usd,cash_margin,gmv_usd_s,margin,discount,offer_name
0,-263518,[-201673],[Pasta Espagueti La Moderna x 200 gr],[293082.6875],31946.012937,293082.687500,10.900000,10.00,SH_AOV_VIEW_CMX_2023-07-12_-263518
1,-263753,"[-201441, 229750]","[Champiñón Blanco, Tortilla de Maíz Ochoa x 80...","[142611.828125, 19387.970703125]",15114.253523,161999.796875,9.329798,6.53,SH_AOV_VIEW_CMX_2023-07-12_-263753


In [88]:
csv_file = {} 
csv_file['offer_name'] = result_df.offer_name
csv_file['offer_description'] = None
csv_file['discount'] = result_df.discount
csv_file['automatically_consider_offer'] = "true"
csv_file['start_date'] = (datetime.datetime.today() + datetime.timedelta(days=dias)).strftime('%d-%m-%Y')
csv_file['end_date'] = (datetime.datetime.today() + datetime.timedelta(days=(dias+1))).strftime('%d-%m-%Y')
csv_file['max_uses_per_order'] = 0
csv_file['max_uses_per_customer'] = 1
csv_file['customer_segment_id'] = [1]*len(result_df.offer_name) #[None]
csv_file['sku_id'] = result_df['card_id'].apply(concatenate_skus)

global_offer_csv = pd.DataFrame(csv_file) #, index=[0])

In [89]:
global_offer_csv.head()

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,SH_AOV_VIEW_CMX_2023-07-12_-263518,None,10.00,true,12-07-2023,13-07-2023,0,1,1,136394#136396#136392
1,SH_AOV_VIEW_CMX_2023-07-12_-263753,None,6.53,true,12-07-2023,13-07-2023,0,1,1,-304515#113620#290230
2,SH_AOV_VIEW_CMX_2023-07-12_-267792,None,10.00,true,12-07-2023,13-07-2023,0,1,1,143466#395045#213232#-304617#70550#-304614#705...
3,SH_AOV_VIEW_CMX_2023-07-12_-267812,None,10.00,true,12-07-2023,13-07-2023,0,1,1,-305196
4,SH_AOV_VIEW_CMX_2023-07-12_-268150,None,10.00,true,12-07-2023,13-07-2023,0,1,1,287403#287401#287400#287402


In [90]:
global_segment_list = []
for i in result_df.values:
    global_segment_list.append({'name':i[-1], 'customersIds': [i[0]]})

In [91]:
import requests
import json

url = f"https://{city}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

payload = json.dumps(global_segment_list)


headers = {
  'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

In [92]:
sgt = [(int(x[:6]),x[7:]) for x in list(map(str.strip, response.text[32:].strip('][').replace('"', '').split(',')))]

In [93]:
for i in global_offer_csv.offer_name:
    for j,z in sgt:
        if z==i:
            global_offer_csv.loc[global_offer_csv.offer_name == i, ('customer_segment_id')] = j
            break

In [96]:
global_offer_csv.head()

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,SH_AOV_VIEW_CMX_2023-07-12_-263518,None,10.00,true,12-07-2023,13-07-2023,0,1,188513,136394#136396#136392
1,SH_AOV_VIEW_CMX_2023-07-12_-263753,None,6.53,true,12-07-2023,13-07-2023,0,1,188514,-304515#113620#290230
2,SH_AOV_VIEW_CMX_2023-07-12_-267792,None,10.00,true,12-07-2023,13-07-2023,0,1,188515,143466#395045#213232#-304617#70550#-304614#705...
3,SH_AOV_VIEW_CMX_2023-07-12_-267812,None,10.00,true,12-07-2023,13-07-2023,0,1,188516,-305196
4,SH_AOV_VIEW_CMX_2023-07-12_-268150,None,10.00,true,12-07-2023,13-07-2023,0,1,188517,287403#287401#287400#287402


In [94]:
#//--------------------------------------------
#//Exporting single file data
global_offer_csv.to_csv(f'SD/export_data/Below_Media/{city}/{todays_date}_SH_AOV_VIEW_{city}_OFFER_csv.csv', index = False)

In [95]:
result_df.to_excel(f'SD/export_data/Below_Media/{city}/{todays_date}_SH_RESULTS_AOV_VIEW_{city}_OFFER_csv.xlsx', index = False)